In [74]:
import random

In [75]:


class RSAReciever:
    def __init__(self, bit_len):
        self.bit_len = bit_len

        # generamos p y q aleatorios de bit_len
        self.p = self._generar_primo(bit_len)
        self.q = self._generar_primo(bit_len)

        # nos aseguramos que p != q
        while self.p == self.q:
            self.q = self._generar_primo(bit_len)
        
        # generamos las llaves
        self._generate_keys()

    def get_public_key(self):
        
        # generamos la llave publica con los numeros e y n
        e_bytes = self._int_to_bytes(self.e)
        n_bytes = self._int_to_bytes(self.n)
        number_e_bytes = len(e_bytes).to_bytes(4,'big')
        number_n_bytes = len(n_bytes).to_bytes(4,'big')
        return number_e_bytes + e_bytes + number_n_bytes + n_bytes

    def decrypt(self, ciphertext):

        mensaje_decodificado = b''

        for i in range(0, len(ciphertext), self.largo_bloque+1):
            largo = self.largo_bloque+1
            """ if i+largo > len(message):
                largo = bytes_mensaje % self.largo_bloque """
            bloque = ciphertext[i:i+largo]
            # decodificamos cada mensaje con la exponenciacion rapida de C ** D Mod N
            bloque = int.from_bytes(bloque, 'big')
            bloque = pow(bloque, self.d, self.n)
            bloque = bloque.to_bytes(length=largo+1, byteorder='big')
            mensaje_decodificado += bloque

        # pasamos el mensaje a texto
        return mensaje_decodificado.decode('utf-8').replace('\x00','')
    
    def _generate_keys(self):

        # calculamos N
        self.n = self.p * self.q

        # calculamos n
        self.largo_bloque = len(self._int_to_bytes(self.n)) - 1

        # calculamos phi
        self.phi = (self.p-1) * (self.q-1)

        # sacamos un e aleatorio con el MCD
        self.e = random.randint(2, self.phi - 1)
        while self._gcdExtended(self.e, self.phi)[0] != 1:
            self.e = random.randint(2, self.phi - 1)

        # sacamos el inverso de e
        self.d = self._inverso(self.e, self.phi)

        # generamos las llaves
        self.public_key = (self.e, self.n)
        self.private_key = (self.d, self.n)
    
    # https://stackoverflow.com/questions/21017698/converting-int-to-bytes-in-python-3
    def _int_to_bytes(self,x: int) -> bytes:
        return x.to_bytes((x.bit_length() + 7) // 8, 'big')
    
    def _gcdExtended(self, a, b): 
        # https://www.geeksforgeeks.org/python-program-for-basic-and-extended-euclidean-algorithms-2/
        if a == 0 :  
            return b,0,1
                
        gcd,x1,y1 = self._gcdExtended(b%a, a) 

        x = y1 - (b//a) * x1 
        y = x1 
        
        return gcd,x,y

    def _inverso(self,a, m):
        # algoritmo para sacar el inverso modular
        g, x, y = self._gcdExtended(a, m)
        if g != 1:
            raise Exception('modular inverse does not exist')
        else:
            return x % m
    
    def _generar_primo(self, bit_size):
        # algoritmo sacado de internet para generar primos aleatorios de bit size

        # Sacamos un entero aleatorio de bit size y verificamos si es primo
        p = random.getrandbits(bit_size)
        if not p & 1:
            p += 1
        while not self._miller_rabin(p):  # Test de primalidad
            p = p + 2
        return p
    
    def _miller_rabin(self, n, k=10):
        # test miller rabin visto en diseño y analisis de algoritmos con k=10
        if n == 2:
            return True
        if n == 1 or n % 2 == 0:
            return False
        
        primos = [3,5,7,11,13,17,19,23,29,31,37,41]

        for primo in primos:
            if n % primo == 0:
                return False

        s, d = 0, n - 1
        while not d % 2:
            s += 1
            d //= 2
        assert(2**s*d == n - 1)

        def check_if_composite_using(a):
            x = pow(a, d, n)
            if x == 1 or x == n - 1:
                return False 
            for _ in range(s):
                x = (x * x) % n
                if x == n - 1:
                    return False
            return True 
        for _ in range(k):
            a = random.randint(2, n-1)
            if check_if_composite_using(a):
                return False 
        return True 


In [76]:
class RSASender:
    def __init__(self, public_key):
        self.public_key = public_key
        # sacamos N, e y n
        self.numero_e, self.numero_N, self.largo_bloque = self._getNandE()
    
    def encrypt(self, message):
        message = bytearray(message,'utf-8')
        bytes_mensaje = len(message)
        
        mensaje_codificado = b''

        for i in range(0,len(message),self.largo_bloque):
            largo = self.largo_bloque
            """ if i+largo > len(message):
                largo = bytes_mensaje % self.largo_bloque """ 
            bloque = message[i:i+largo]
            # encriptamos cada mensaje con la exponenciacion rapida de m ** e Mod N
            bloque = int.from_bytes(bloque,'big')
            bloque = pow(bloque,self.numero_e,self.numero_N)
            bloque = bloque.to_bytes(length=largo+1,byteorder='big')
            mensaje_codificado += bloque
            
        return mensaje_codificado
        
    
    def _getNandE(self):
        # separamos el byte array
        actual = 0
        largo_e = self.public_key[actual:4]
        actual += 4
        largo_e = int.from_bytes(largo_e,'big')
        numero_e = self.public_key[actual:actual+largo_e]
        numero_e = int.from_bytes(numero_e,'big')
        actual = actual+largo_e
        largo_N = self.public_key[actual:actual+4]
        actual += 4
        largo_N = int.from_bytes(largo_N,'big')
        numero_N = self.public_key[actual:actual+largo_N]
        
        # n va a ser el numero de bytes de N - 1
        largo_bloque = (largo_N - 1)
        numero_N = int.from_bytes(numero_N,'big')

        return numero_e, numero_N,largo_bloque

    
    
        